# User Interface Introduction

## Demo App has three tabs.
### 1. __New Ingredients__

* **Purpose: Shows new ingredient to the market. (Focus- Last Metadata Scrape)**
* Disclaimer-1: Currenly the lanugage model(machine learning for nlp) used for ingredient data tokenization(separation) is at early stage so there will be some instances of ingredient that might look incomplete. 
* Disclaimer-2: Ingredient type classification is being done with rule based labelling functions which might not be very accurate. A ML based approach is currently under development that will take over labelling functions for much more accurate ingredient type classification.
* **Logic: Take all the ingredients of products that are marked as _"New"_ in e-commerce site. Then check whether any of these ingredients already exist in the old products. If does not exist, mark the ingredient as new.**
* **Interaction: Data is shown in an excel like grid. Apart from two top level filters users can interact with the grid by clicking sorting buttons or excel like filtering buttons. These buttons work exactly as they do in excel.**

<img src="filter.PNG">


### 2. __Ingredient Over Time__
* **Pupose: Shows in how many differect products the new ingredient is used over time.**
* **Disclaimer: As we have only two time points now, Bar chart is used but once we multiple timepoints the charts will be changed to Line chart over time.**

### 3. __Ingredient Type Over Time__
* **Pupose: Shows how the ingredient type usage vary over time.**
* **Disclaimer: As we have only two time points now, Bar chart is used but once we multiple timepoints the charts will be changed to Line chart over time.**

# Develop New Products

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [14]:
import qgrid, pandas as pd, numpy as np, swifter, warnings, os, seaborn as sns
from pathlib import Path
import matplotlib.pyplot as plt
import ipywidgets as ipw
from ipywidgets.widgets import interact, interactive
from IPython.display import display


plt.style.use('ggplot')
# os.getcwd(); os.chdir(r'C:\Amit\Meiyume\voila_apps')
warnings.simplefilter(action='ignore')

In [5]:
ing_df = pd.read_csv('develop_new_app_demo_data.csv')

In [6]:
### Prepare Data
new_ing_now = ing_df[(ing_df.new_flag=='New_Ingredient')&(ing_df.meta_date==ing_df.meta_date.max())]
new_ing_now.drop_duplicates(inplace=True)
new_ing_now.reset_index(drop=True, inplace=True)

nidf = ing_df[['meta_date', 'ingredient', 'prod_id']][(ing_df.new_flag=='New_Ingredient')].groupby(by=['meta_date', 'ingredient']).count().reset_index()
nidf.columns = ['meta_date', 'ingredient', 'count']
nidf['meta_month'] = pd.to_datetime(nidf.meta_date).dt.strftime('%b-%Y')
nidf.set_index('ingredient', inplace=True)
ing_df.set_index('ingredient', inplace=True)
nidf = nidf.join(ing_df['ingredient_type'], how='inner').reset_index().drop_duplicates(subset=['meta_month', 'ingredient'])
ing_df.reset_index(inplace=True)

itdf = ing_df[['meta_date', 'ingredient_type', 'ingredient']].drop_duplicates(subset=['meta_date','ingredient'])\
.groupby(by=['meta_date','ingredient_type']).count().reset_index()
itdf.columns = ['meta_date', 'ingredient_type', 'count']
itdf['meta_month'] = pd.to_datetime(itdf.meta_date).dt.strftime('%b-%Y')

In [7]:
def categorical_unique_sorted_values_all(series):
    selection_list = series.dropna().unique().tolist()
    selection_list.sort()
    selection_list.insert(0, 'All')
    return selection_list

def number_slider_values(series, get:[str]):
    if get=='min': return series.min()
    elif get=='max': return series.max()
    
def colour_value(value, comparison):
    if value >= comparison:
        return 'color: green'
    else:
        return 'color: red'

In [8]:
# create all widgets
dropdown_ing_type = ipw.Dropdown(
    options= categorical_unique_sorted_values_all(ing_df.ingredient_type),
    value='All',
    description='IngType:', width='auto'
)
dropdown_ing_type_plot = ipw.Dropdown(
    options= categorical_unique_sorted_values_all(ing_df.ingredient_type),
    value='All',
    description='IngType:', width='auto'
)
dropdown_prod_type = ipw.Dropdown(
    options= categorical_unique_sorted_values_all(ing_df.product_type[ing_df.new_flag=='New_Ingredient']),
    value='All',
    description='ProductType:', width='auto'
)
dropdown_ingredient_new = ipw.Dropdown(
    options= categorical_unique_sorted_values_all(ing_df.ingredient[ing_df.new_flag=='New_Ingredient']),
    value= nidf.ingredient[0],
    description='ProductType:', width='auto'
)
floatslider_bayes = ipw.FloatSlider(value=4,
                         min=number_slider_values(ing_df.bayesian_estimate, 'min'),
                         max=number_slider_values(ing_df.bayesian_estimate, 'max')+1,
                         description='BayesianScore',
                         continuous_update=False,
                         orientation='horizontal',
                         readout=True)

In [9]:
new_ing_output = ipw.Output(layout={'border': '1px solid black'})

def filter_dataframe(ing_type, prod_type, df = new_ing_now):
    new_ing_output.clear_output(wait=True)
    df = df[['ingredient', 'ingredient_type', 'clean_flag', 'ban_flag', 'product_name', 'bayesian_estimate', 'category', 'product_type']]
    if (ing_type=='All') & (prod_type=='All'):
        filtered_df = df
    elif prod_type=='All':
        filtered_df = df[(df.ingredient_type==ing_type)]
    elif ing_type=='All':
        filtered_df = df[(df.product_type==prod_type)]
    else:
        filtered_df = df[(df.ingredient_type==ing_type)&(df.product_type==prod_type)]
        
    with new_ing_output:        
        df_qgrid = qgrid.show_grid(filtered_df)
        df_qgrid.layout = ipw.Layout(width='1450px')
        display(df_qgrid)

def dropdown_ing_type_eventhandler(change): filter_dataframe(change.new, dropdown_prod_type.value)
def dropdown_prod_type_eventhandler(change): filter_dataframe(dropdown_ing_type.value, change.new)
    
dropdown_ing_type.observe(dropdown_ing_type_eventhandler, names='value')
dropdown_prod_type.observe(dropdown_prod_type_eventhandler, names='value')

In [10]:
plot_ing_time_output = ipw.Output(layout={'border': '1px solid black'})

def plot_ing_over_time(ingredient, df=nidf):
    plot_ing_time_output.clear_output(wait=True)
    
    if ingredient=='All': 
        with plot_ing_time_output:
            print('choose one ingredient at a time')
    else: 
        ing_type = nidf.ingredient_type[nidf.ingredient==ingredient].values[0]
        if ing_type is np.nan:
            ing_type = ''
            
        fig, ax = plt.subplots()
        p = nidf[nidf.ingredient==ingredient].plot.barh(x='meta_month', y='count', figsize=(14,6), ax=ax)
        ax.set_ylabel('DateTime', fontsize=16, color='black')
        ax.set_xlabel('Presence in Products', fontsize=16, color='black')
        plt.xticks(fontsize=12, color='b')
        plt.yticks(fontsize=12, color='b')
        ax.legend((ingredient + ' - ' + ing_type, ),loc='upper center', fontsize=16)
        plt.title('New Ingredient Usage Over Time', fontsize=20)

        with plot_ing_time_output:    
            plt.tight_layout()
            plt.show(p)

def dropdown_ingredient_new_eventhandler(change): plot_ing_over_time(change.new, dropdown_prod_type.value)
dropdown_ingredient_new.observe(dropdown_ingredient_new_eventhandler, names='value')

In [11]:
plot_ingtype_time_output = ipw.Output(layout={'border': '1px solid black'})

def plot_ingtype_over_time(ing_type, df=itdf):
    plot_ingtype_time_output.clear_output(wait=True)
    
    if ing_type=='All': df = itdf
    else: df = itdf[itdf.ingredient_type==ing_type]
    with sns.axes_style({'ytick.color':'b','xtick.color':'b', 
                         'font_scale':1.2, 'style':"darkgrid"}):
        
        fig, ax = plt.subplots(1,1, figsize=(12,6))
        g = sns.catplot(x="count", y="meta_month", hue="ingredient_type", 
                        data=df, kind="bar", palette="muted", ax=ax)
        g.despine(left=True)
        ax.set_title('Ingredient Type Over Time', fontsize=20)
        ax.set_ylabel('DateTime', fontsize=16, color='black')
        ax.set_xlabel('Total Identified Ingredients', fontsize=16, color='black')
        plt.setp(ax.get_legend().get_texts(), fontsize='14') # for legend text
        plt.setp(ax.get_legend().get_title(), fontsize='16')
        # Put the legend out of the figure
        ax.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
        if ing_type!='All':
            for i, v in enumerate(df['count'].values):
                ax.text( v, i  , str(v), color='black', fontweight='bold')
        
        with plot_ingtype_time_output:
            plt.close(g.fig)
            plt.tight_layout()
            plt.show(g)

def dropdown_ing_type_time_eventhandler(change): plot_ingtype_over_time(change.new, dropdown_prod_type.value)
dropdown_ing_type_plot.observe(dropdown_ing_type_time_eventhandler, names='value')

In [12]:
new_ing_widgets = ipw.VBox([ipw.HBox([dropdown_ing_type, dropdown_prod_type]), new_ing_output])
plot_ing_time_widgets = ipw.VBox([dropdown_ingredient_new, plot_ing_time_output])
plot_ingtype_time_widgets = ipw.VBox([dropdown_ing_type_plot, plot_ingtype_time_output])

tab = ipw.Tab([new_ing_widgets, plot_ing_time_widgets,plot_ingtype_time_widgets])
tab.set_title(0, 'New Ingredients')
tab.set_title(1, 'Ingredient Over Time')
tab.set_title(2, 'IngredientType Over Time')
tab.layout = ipw.Layout(width='1500px')

In [13]:
display(tab)
dropdown_ing_type.value = 'food'
dropdown_ing_type.value = 'All'
dropdown_ingredient_new.value = nidf.ingredient.values[2]
dropdown_ing_type_plot.value = 'food'
# dropdown_ingredient_new.value = nidf.ingredient.values[1]